In [1]:
## https://cloud.gate.ac.uk/yourAccount/newApiKey
#Key ID   == gcc3g8cmwyob
#Password == 8ypn2huuqhu43zm3i2in
#Description == Generated at 2020-09-22T12:05:02.182Z

###resultFile-BBC-bbccouknewsuk-northern-ireland-54245790

In [6]:
import requests
#url = "https://www.bbc.co.uk"
#url = "https://metro.co.uk/" ## 82.5 ### 'Left Center' ## 'Media Bias/Fact Check' ## 20191001
#url = "https://www.thesun.co.uk/" ##95
#url = "https://socialistworker.co.uk/" ## null
#url = "https://www.spiked-online.com/" ## null
#url = "https://www.thecanary.co/"## null
#url = "https://www.breitbart.com/europe/" ###'credibility-category': 'Fake News',
#url= "http://foodbabe.com/"

#url = "https://www.bbc.co.uk"
url = "https://www.usatoday.com"

Gate_Source_Cred_API = "https://cloud-api.gate.ac.uk/process-document/source-credibility?annotations=:DomainCredibility"
reliability_txt = requests.post(Gate_Source_Cred_API, data = url, headers={'Content-Type': 'text/plain'}, 
                                auth=('gcc3g8cmwyob', '8ypn2huuqhu43zm3i2in'))
rel_json = reliability_txt.json()
rel_json

{'text': 'https://www.usatoday.com', 'entities': {}}

In [193]:
scores = []
for item in rel_json['entities']['DomainCredibility']:
    if item['credibility-source'] == 'Media Bias/Fact Check': ###!= 'NewsTrust':
        scores += [item['credibility-score']]

#scores = [item['credibility-score'] for item in rel_json['entities']['DomainCredibility']]

import statistics 
avg_score = statistics.mean(scores)
print("Mean is :", avg_score)

Mean is : 95.0


In [1]:
## Function: getCredibilityScore
## Input: URL
## Output: Credibility Score, Category and Source
## Author: The Sentimentalists / Ana B Potje
## Date: 24/Sep/2020
##
## This function calls the API Gate Source Credibility passing the URL to be analized.
## It returns the Score, Category (Left Center, Fake News, ...) and the Source.
## If the website is reated in the source 'Media Bias/Fact Check', this source is then used.
## Otherwise it will a different source that rated the website, or return "-1" in case no rating is found.
###########################################################################################################

def getCredibilityScore(url):
    import requests
    import getSecret as secrets

    p_score = 0
    p_category = '' 
    p_source = ''
    return_dict = {'type': 'credibility', 'outcome': {'score': -1} }

    aws_secrets = secrets.getSecret("prod/getCredibilityScore/GATEKey", "eu-west-2")
    if "error" in aws_secrets:
        return return_dict
    
    Gate_Source_Cred_API = "https://cloud-api.gate.ac.uk/process-document/source-credibility?annotations=:DomainCredibility"
    cred_txt = requests.post(Gate_Source_Cred_API, data = url, headers={'Content-Type': 'text/plain'}, 
                                    auth=(aws_secrets['apiid'], aws_secrets['apikey']))
    rel_json = cred_txt.json()

    if (rel_json['entities'] != {}):
        for item in rel_json['entities']['DomainCredibility']:
            p_score = item.get('credibility-score', 0)
            p_category = item.get('credibility-category', '')
            p_source = item.get('credibility-source', '')

            return_dict = {'type': 'credibility', 'outcome' : { 'score': p_score, 'source': p_source, 'category': p_category }}

            if item['credibility-source'] == 'Media Bias/Fact Check':
                return(return_dict)
    else:
        return_dict = {'type': 'credibility', 'outcome': {"error" : "The credibility score was not available."}}
        return(return_dict)
    
    ## in case there is no 'Media Bias/Fact Check' but a different source:    
    return(return_dict)

In [6]:
#url = "https://metro.co.uk/"              ## (82.5, 'Left Center', 'Media Bias/Fact Check')
#url = "https://www.thesun.co.uk/"         ## (95.0, 'UNS', , 'Media Bias/Fact Check')      ### RIGHT BIAS
#url = "http://foodbabe.com/"               ## (25.0, 'unreliable', 'Human Assigned')
#url = "http://www.mercola.com/"           ## (35.0, 'Conspiracy', 'Media Bias/Fact Check')
#
#url = "https://www.breitbart.com/europe/" ## (49.5, 'Fake News', 'Media Bias/Fact Check')
#url = "https://socialistworker.co.uk/"    ## -1
#url = "https://www.spiked-online.com/"    ## -1
#url = "https://www.thecanary.co/"         ## -1

url = "https://www.bbc.co.uk"             ## (95.0, 'UNS', 'Media Bias/Fact Check') ### LEFT-CENTER BIAS
#url = "https://www.bbc.co.uk/news/election-us-2020-54350538"

#url = "https://metro.co.uk/"              ## (82.5, 'Left Center', 'Media Bias/Fact Check')
#url = "https://eu.usatoday.com/"
#url = 'https://www.wsj.com/articles/covid-19-left-u-k-s-boris-johnson-and-brazils-jair-bolsonaro-unscathed-politically-11601739583'
#url = "https://www.nbcnews.com/news/us-news/canadian-woman-charged-sending-ricin-laced-letter-white-house-n1240760"

#url = 'http://www.alternativenation.net/van-halen-fans-detail-tour-new-guitarist-proposal/'
#url = 'http://www.angelfire.lycos.com/'

getCredibilityScore(url)

{'type': 'credibility',
 'outcome': {'score': 95.0,
  'source': 'Media Bias/Fact Check',
  'category': 'UNS'}}

In [ ]:
url = "https://www.thesun.co.uk/"         ## (95.0, 'UNS', , 'Media Bias/Fact Check')      ### RIGHT BIAS
url = "https://www.bbc.co.uk"             ## (95.0, 'UNS', 'Media Bias/Fact Check')        ### LEFT-CENTER BIAS


In [9]:
## Function: getCredibilityScore
## Input: URL
## Output: Credibility Score, Category and Source
## Author: The Sentimentalists / Ana B Potje
## Date: 24/Sep/2020
##
## This function calls the API Gate Source Credibility passing the URL to be analized.
## It returns the Score, Category (Left Center, Fake News, ...) and the Source.
## If the website is reated in the source 'Media Bias/Fact Check', this source is then used.
## Otherwise it will a different source that rated the website, or return "-1" in case no rating is found.
###########################################################################################################

def getCredibilityScore(url):
    import requests
#    import getSecret as secrets

    p_score = 0
    p_category = '' 
    p_source = ''
    return_dict = {'type': 'credibility', 'outcome': {'score': -1} }

    return_dict = {'type': 'credibility', 'outcome' : { 'score': 50, 'source': 'media bias', 'category': 'fake news' }}

    
    ## in case there is no 'Media Bias/Fact Check' but a different source:    
    return(return_dict)

In [12]:
getCredibilityScore
url = "https://www.bbc.co.uk/news/election-us-2020-54350538"
x = getCredibilityScore(url)
type(x)

dict

In [16]:
x
if 'score' in x['outcome']:
    print('ok')

outcome


ok


{'score': 50, 'source': 'media bias', 'category': 'fake news'}

In [8]:
def getBiasScore(credibility, polarity, subjectivity):
            
    IncredibilityPercent = (100-credibility)
    PolarityPercent = (polarity)*100
    SubjectivityPercent = (subjectivity)*100
    
    if (credibility >= 0):
        BiasIndicator = ((IncredibilityPercent + abs(PolarityPercent) + SubjectivityPercent)/3)
    else:
        BiasIndicator = ((abs(PolarityPercent)) + SubjectivityPercent)/2

    response = {'type':'bias','outcome':{'score':BiasIndicator}}
    return response


In [ ]:
import getBiasScore as bs

    if 'error' in return_dict['outcome']:
        cred_input = -1
    else:
        cred_input = return_dict['outcome']['score']

    try:
        biasscoreresult = bs.getBiasScore(cred_input,      )
        object['results'].append(credibilityresult)
    except Exception as e:
        object['results'].append({'type': 'credibility', 'outcome': {"error" : "The credibility score was not available."}})


In [44]:
#return_dict = {'type': 'credibility', 'outcome' : { 'score': 90}}#, 'source': p_source, 'category': p_category }}
return_dict = {'type': 'credibility', 'outcome': {"error" : "The credibility score was not available."}}


In [33]:
#return_dict['outcome']['error']
#return_dict['outcome']['score']


'The credibility score was not available.'

In [45]:
if 'error' in return_dict['outcome']:
    cred_input = -1
else:
    cred_input = return_dict['outcome']['score']

cred_input

#if credibilityresult['text'] == '-1':

-1

In [2]:
from aws_xray_sdk.core import xray_recorder
from aws_xray_sdk.core import patch


In [3]:
from aws_xray_sdk.core import xray_recorder
import sys
import logging
import traceback
from newspaper import Article 
import nltk

## Function: getText
## Input: URL
## Output: Unformatted TEXT extracted from the URL and ARTICLE
## Author: The Sentimentalists / Ana B Potje
## Date: 24/Sep/2020
## Last Modified: 22/Oct/2020
##
## This function uses the Python Library Newspaper, which reads the article from the URL,
## remove some unwanted characters and words, so the unformatted text is stored in a variable "text".
## It then applies the "nlp()" function to perform natural language processing on the news article.
##
## The objects "text" and "article" are the output of this function.
##
## PS: Use of the "article" object:
##   news_article = getArticle("https://www.bbc.co.uk/news/uk-54234084")
##   news_article.top_image
##   >>> https://ichef.bbci.co.uk/images/ic/1024x576/p08s2yp9.jpg
##   news_article.keywords
##   >>> ['action', '50000', 'patrick', 'face', 'covid19', 'restrictions', 'governments', ...]
##   news_article.summary
##   >>> Sir Patrick Vallance said that would be expected to lead to about "200-plus deaths per day" a 
##       month after that. On Monday, a further 4,368 daily cases were reported in the UK, up from 3,899....
###########################################################################################################

logger = logging.getLogger()
logger.setLevel(logging.INFO)
nltk.data.path.append("./nltk_data")

@xray_recorder.capture('getText')
def getText(url):
    
    logger.info(f'getText: initialised Article and ntlk')
    ### Getting the ARTICLE
    try:
        print("getText: Initialising Article...")
        subsegment = xray_recorder.begin_subsegment('getText: init article')
        article = Article(url)
        xray_recorder.end_subsegment()

        print("getText: Downloading Article...")
        subsegment = xray_recorder.begin_subsegment('getText: download article')
        article.download()
        xray_recorder.end_subsegment()

        print("getText: Parsing Article...")
        subsegment = xray_recorder.begin_subsegment('getText: parse article')
        article.parse()
        xray_recorder.end_subsegment()
    ### Exception - e.g if URL is "valid" but inexistent, no text will be retrieved
    except Exception as e: 
        print("getText: Exception: " + str(e))
        return  {'text': '-1',
                 'header': '', 
                 'summary': '',
                 'keywords':''}

    subsegment = xray_recorder.begin_subsegment('getText: nlp article')
    article.nlp()
    xray_recorder.end_subsegment()
    text = article.text

    ### Removing unwanted formatting
    text = text.replace("\n\n", "")
    text = text.replace("Image copyright Getty Images ", "")
    text = text.replace("Image copyright Getty Images/Reuters", "")
    text = text.replace("Image caption ", "")
    text = text.replace("Media playback is unsupported on your device ", "")
    text = text.replace("Media caption ", "")


    return  {'text': text,
             'header': article.title, 
             'summary': article.summary,
             'keywords': article.keywords}


In [4]:
url = "https://www.bbc.co.uk/news/election-us-2020-54350538"
getText(url)


cannot find the current segment/subsegment, please make sure you have a segment open


SegmentNotFoundException: cannot find the current segment/subsegment, please make sure you have a segment open